### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.9 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.1/366.1 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 k

In [2]:
!pip install --upgrade --quiet  lark qdrant-client

### Load CSV data from S3

In [3]:
!pwd

/home/ec2-user/SageMaker/Enterprise-RAG/notebooks


In [4]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,Name,RecipeCategory,Description,RecipeIngredientParts,RecipeInstructions,Keywords_string,AggregatedRating,ReviewCount,Combined_Features,Combined_Features_Clean
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,Make and share this Low-Fat Berry Blue Frozen ...,"[blueberries, granulated sugar, vanilla yogurt...",Toss 2 cups berries with sugar. Let stand for ...,Dessert Low Protein Low Cholesterol Healthy Fr...,4.5,4.0,Low-Fat Berry Blue Frozen Dessert Frozen Desse...,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,Biryani,Chicken Breast,Make and share this Biryani recipe from Food.com.,"[saffron, milk, hot green chili peppers, onion...",Soak saffron in warm milk for 5 minutes and pu...,Chicken Thigh & Leg Chicken Poultry Meat Asian...,3.0,1.0,Biryani Chicken Breast Make and share this Bir...,Biryani Chicken Breast Make share Biryani reci...
2,Best Lemonade,Beverages,This is from one of my first Good House Keepi...,"[sugar, lemons, rind of, lemon, zest of, fresh...","Into a 1 quart Jar with tight fitting lid, put...",Low Protein Low Cholesterol Healthy Summer < 6...,4.5,10.0,Best Lemonade Beverages This is from one of my...,Best Lemonade Beverages one first Good House K...
3,Carina's Tofu-Vegetable Kebabs,Soy/Tofu,This dish is best prepared a day in advance to...,"[extra firm tofu, eggplant, zucchini, mushroom...","Drain the tofu, carefully squeezing out excess...",Beans Vegetable Low Cholesterol Weeknight Broi...,4.5,2.0,Carina's Tofu-Vegetable Kebabs Soy/Tofu This d...,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,Cabbage Soup,Vegetable,Make and share this Cabbage Soup recipe from F...,"[plain tomato juice, cabbage, onion, carrots, ...",Mix everything together and bring to a boil. R...,Low Protein Vegan Low Cholesterol Healthy Wint...,4.5,11.0,Cabbage Soup Vegetable Make and share this Cab...,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['Name', 'RecipeCategory', 'Description', 'RecipeIngredientParts',
       'RecipeInstructions', 'Keywords_string', 'AggregatedRating',
       'ReviewCount', 'Combined_Features', 'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

"Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Toss 2 cups berries sugar. Let stand 45 minutes, stirring occasionally. Transfer berry-sugar mixture food processor. Add yogurt process smooth. Strain fine sieve. Pour baking pan (or transfer ice cream maker process according manufacturers' directions). Freeze uncovered edges solid centre soft. Transfer processor blend smooth again. Return pan freeze edges solid. Transfer processor blend smooth again. Fold remaining 2 cups blueberries. Pour plastic mold freeze overnight. Let soften slightly serve. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy"

In [8]:
df = df[df['AggregatedRating'].notna()]
df.shape

(269294, 10)

### Load data into chunked documents

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [10]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Note: chunking may not be useful for us as the EDA has the following about description length
print(new_recipe['Combined_Features_Clean'].str.len().describe())


count    522517.000000\
mean        211.461774\
std         122.388646\
min          44.000000\
25%         131.000000\
50%         180.000000\
75%         255.000000\
max        4174.000000\
Name: Combined_Features_Clean, dtype: float64

In [11]:
def create_textsplitter(chunks, overlaps):
    splits = {}
    for chunk in chunks:
        for overlap in overlaps:
            splits[f'chunk{str(chunk)}_overlap{str(overlap)}'] = RecursiveCharacterTextSplitter(chunk_size=chunk, chunk_overlap=overlap)
    return splits

In [12]:
chunks = [128, 256, 512, 1024, 2048]
overlaps = [.25, .2, .15, .1]

text_splits = create_textsplitter(chunks, overlaps)

In [13]:
text_splits['chunk1024_overlap0.25']

In [16]:
# def create_documents(df):
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             # 'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
#             'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
#             # 'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
#             # 'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
#             # 'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
#             'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
#             # 'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
#             'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
#             'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
#             # 'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
#             # 'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
#             # 'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
#             # 'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
#             # 'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
#             # 'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
#             # 'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
#             # 'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
#             # 'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
#             # 'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
#         }

#         # Use Combined_Features_Clean for the document content
#         text = str(row['Combined_Features_Clean'])
#         doc = Document(page_content=text, metadata=metadata)
#         documents.append(doc)
        
#     return documents

def create_documents(df):
    df_copy = df.copy(deep=True)
    df = df.dropna(subset=["AggregatedRating"])
    df_copy = df_copy.fillna("")  # Convert NA values to empty strings
    df_copy = df_copy.astype(str)  # Cast all columns to string

    documents = []
    for _index, row in df_copy.iterrows():
        metadata = {
            "name": row["Name"] if row["Name"] else "No Name Available",
            "description": (
                row["Description"] if row["Description"] else "No Description Available"
            ),
            "recipe_category": (
                row["RecipeCategory"]
                if row["RecipeCategory"]
                else "No Category Available"
            ),
            "keywords": (
                row["Keywords_string"]
                if row["Keywords_string"]
                else "No Keywords Available"
            ),
            "recipe_ingredient_parts": (
                row["RecipeIngredientParts"]
                if row["RecipeIngredientParts"]
                else "No Recipe Ingredient Parts Available"
            ),
            "recipe_instructions": (
                row["RecipeInstructions"]
                if row["RecipeInstructions"]
                else "No Recipe Instructions Available"
            ),
            "aggregated_rating": (
                row["AggregatedRating"]
                if row["AggregatedRating"]
                else "No Rating Available"
            ),
            "review_count": (
                row["ReviewCount"] if row["ReviewCount"] else "No Reviews Available"
            ),
        }

        # List of fields to be included in the document content
        content_field = (
            row["Combined_Features"]
            if row["Combined_Features"]
            else "No Content Available"
        )

        # Create the document content using the combined features field
        doc = Document(page_content=content_field, metadata=metadata)
        documents.append(doc)

    return documents


In [17]:
# Take a sample of 100K receipes 
documents = create_documents(df.sample(n=10000))

In [18]:
documents[0]

Document(metadata={'name': 'Cherry Chicken Salad', 'description': 'The original recipe is from TOH with some additions by me.  We had this for lunch today and really enjoyed it.  Cook time is refrigeration time.', 'recipe_category': 'Lunch/Snacks', 'keywords': 'Chicken Breast Chicken Poultry Meat < 4 Hours', 'recipe_ingredient_parts': "['dried cherries' 'celery' 'red onion' 'tart apple' 'hard-boiled egg'\n 'light mayonnaise' 'lemon juice']", 'recipe_instructions': 'In a large bowl, combine the chicken, cherries, celery, onion, apple, egg and sliced almonds. In another small bowl, combine the mayo and lemon juice. Add to chicken mixture and mix well. Season with salt and pepper to taste. Refrigerate for 1 hour to allow flavors to blend. Serve on Croissants or crackers.', 'aggregated_rating': '5.0', 'review_count': '7.0'}, page_content='Cherry Chicken Salad Lunch/Snacks The original recipe is from TOH with some additions by me.  We had this for lunch today and really enjoyed it.  Cook ti

In [19]:
len(documents)

10000

In [20]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [21]:
split_documents = {key: split_documents_with_metadata(documents, text_splitter_recursive) for key, text_splitter_recursive in text_splits.items()}

In [22]:
split_documents['chunk256_overlap0.25'][0]

Document(metadata={'name': 'Cherry Chicken Salad', 'description': 'The original recipe is from TOH with some additions by me.  We had this for lunch today and really enjoyed it.  Cook time is refrigeration time.', 'recipe_category': 'Lunch/Snacks', 'keywords': 'Chicken Breast Chicken Poultry Meat < 4 Hours', 'recipe_ingredient_parts': "['dried cherries' 'celery' 'red onion' 'tart apple' 'hard-boiled egg'\n 'light mayonnaise' 'lemon juice']", 'recipe_instructions': 'In a large bowl, combine the chicken, cherries, celery, onion, apple, egg and sliced almonds. In another small bowl, combine the mayo and lemon juice. Add to chicken mixture and mix well. Season with salt and pepper to taste. Refrigerate for 1 hour to allow flavors to blend. Serve on Croissants or crackers.', 'aggregated_rating': '5.0', 'review_count': '7.0', 'chunk_id': 0}, page_content='Cherry Chicken Salad Lunch/Snacks The original recipe is from TOH with some additions by me.  We had this for lunch today and really enjoy

In [23]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

# From documents with no chunking
qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

# from split documents (specify the chunk and overlap key)
# qdrant_store = Qdrant.from_documents(split_documents['chunk1024_overlap0.25'],
#     embedding_model,
#     location=":memory:",
# )

## Create llm query call

In [24]:
import json

In [25]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [26]:
baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [27]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


In [28]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [29]:
qa_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

You are evaluating for consistency of the user query.
Please retrieve documents that DO NOT violate dietary restrictions, allergies, 
or any requirements dictated by the user.

"""

In [168]:
def qa_query_bedrock_llm(messages):

#     if not isinstance(messages, list):
#         raise ValueError("Messages should be a list of dictionaries")
    print(messages)
    print()
    
    structured_query = {
    "query": "no peanuts",
    "filter": 'not(like("keywords", "peanuts"))'
}
    
    structured_query_json = json.dumps(structured_query, indent=4)
    
    # messages = [messages]
    messages = [
        {"role": "user", "content": "You are a helpful assistant that provides recipes without peanuts."},
        {"role": "assistant", "content": "Recipes with no peanuts"}
    ]
    serializable_messages = []
    for message in messages:
        if isinstance(message, dict) and "role" in message and "content" in message:
            serializable_messages.append(message)
        else:
            # Attempt to convert any other type to a string
            serializable_messages.append({"role": "user", "content": str(message)})

    # messages = [
    # {"role": "user", "content": "You are a helpful assistant that provides recipes without peanuts."},
    # {"role": "assistant", "content": "Recipes with no peanuts"}
    # ]
    
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': qa_sys_prompt,
            'messages': serializable_messages,

            'max_tokens': 3000,
            # "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

## Create Retrieval Chainfrom langchain_core.prompts import ChatPromptTemplate

In [169]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [170]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

In [171]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [172]:
def process_prompt_basic(query_args):
    prompt_with_context = baseline_user_prompt.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

## Create Initial Retriever

In [173]:
qdrant_retriever_rerank = qdrant_store.as_retriever(search_type='mmr', search_kwargs={"k": 50, 'lambda_mult': 0.5})


### Self Query

In [174]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="keywords",
        description="Keywords within the recipe",
        type="string",
    )
]

document_content_description = "List of recipes"
llm = qa_query_bedrock_llm
# qa_retriever = SelfQueryRetriever.from_llm(
#     llm, qdrant_store, document_content_description, metadata_field_info, verbose=True
# )
qa_retriever = SelfQueryRetriever.from_llm(
    llm, qdrant_store, document_content_description, metadata_field_info, verbose=True
)


In [175]:
# print(qa_query_bedrock_llm("Recipes with no peanuts"))

In [176]:
output = qa_retriever.invoke("Recipes with no peanuts")


text='Your goal is to structure the user\'s query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    "query": string \\ text string to compare to document contents\n    "filter": string \\ logical condition statement for filtering documents\n}\n```\n\nThe query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.\n\nA logical condition statement is composed of one or more comparison and logical operation statements.\n\nA comparison statement takes the form: `comp(attr, val)`:\n- `comp` (eq | lt | lte | gt | gte | like): comparator\n- `attr` (string):  name of attribute to apply the comparison to\n- `val` (string): is the comparison value\n\nA logical operation statement takes the form `op(statement1, statement2, ...)`:\n- `op` (and | or 

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

### Cross encoder query

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=
)

compressed_docs = compression_retriever.invoke("What pizza can I make in 20 minutes?")
compressed_docs

### Chain

In [ ]:
qdrant_rag_chain_rerank = (
    RunnableMap(
        {"context": qdrant_retriever_rerank | qa_retriever | format_docs,
         "query": RunnablePassthrough()}
    )
    | process_prompt_basic
    | query_bedrock_llm
)

In [ ]:
qdrant_rag_chain_rerank.invoke("Give me a peanut free Thai dish")